# Imports

In [1]:
import os
import pandas as pd
import random
import time

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from tqdm import tqdm

import ollama

In [2]:
# Force CUDA usage
os.environ["OLLAMA_BACKEND"] = "cuda"
os.environ["OLLAMA_NUM_THREADS"] = "16"

### Parameters

In [3]:
# List of models
ml_models = [
    ('Logistic Regression', LogisticRegression(random_state=42, max_iter=1000)), # 2 sec
    ('Random Forest', RandomForestClassifier(random_state=42)), # 2 min
    ('SVM', SVC(random_state=42)), # 30 min
    ('KNN', KNeighborsClassifier()), # 30 sec
    ('Gradient Boosting', GradientBoostingClassifier(random_state=42)) # 30 sec
]

In [4]:
# LLMs
models = ['llama3.2:1b', 'llama3.2:3b', 'gemma3:1b', 'gemma3:4b', 'llama3.1', 'dolphin3', 'mistral', 'deepseek-llm']

In [5]:
# Vectorize (Bag of words representation)
vectorizer = CountVectorizer(stop_words='english', max_features=10000, ngram_range=(1, 2))

In [6]:
# Number of rows that LLM will generate and compare to
n_rows = 5000

# Load data

In [7]:
# Load the CSV file into a pandas DataFrame
base_df = pd.read_csv('IMB_preprocessed_2025_04_06.csv')

# We want to be able to read the full reviews
pd.set_option('display.max_colwidth', None) 

# Display the first 5 records
base_df.head()

,review,sentiment
0,"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.\nThe first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.\nIt is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.\nI would say the main appeal of the show is due to the fact that it goes where other shows wouldn't dare. Forget pretty pictures painted for mainstream audiences, forget charm, forget romance...OZ doesn't mess around. The first episode I ever saw struck me as so nasty it was surreal, I couldn't say I was ready for it, but as I watched more, I developed a taste for Oz, and got accustomed to the high levels of graphic violence. Not just violence, but injustice (crooked guards who'll be sold out for a nickel, inmates who'll kill on order and get away with it, well mannered, middle class inmates being turned into prison bitches due to their lack of street skills or prison experience) Watching Oz, you may become comfortable with what is uncomfortable viewing....thats if you can get in touch with your darker side.",1
1,"A wonderful little production. \nThe filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. \nThe actors are extremely well chosen- Michael Sheen not only ""has got all the polari"" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master's of comedy and his life. \nThe realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional 'dream' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell's murals decorating every surface) are terribly well done.",1
2,"I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue is witty and the characters are likable (even the well bread suspected serial killer). While some may be disappointed when they realize this is not Match Point 2: Risk Addiction, I thought it was proof that Woody Allen is still fully in control of the style many of us have grown to love.\nThis was the most I'd laughed at one of Woody's comedies in years (dare I say a decade?). While I've never been impressed with Scarlet Johanson, in this she managed to tone down her ""sexy"" image and jumped right into a average, but spirited young woman.\nThis may not be the crown jewel of his career, but it was wittier than ""Devil Wears Prada"" and more interesting than ""Superman"" a great comedy to go see with friends.",1
3,"Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his parents are fighting all the time.\nThis movie is slower than a soap opera... and suddenly, Jake decides to become Rambo and kill the zombie.\nOK, first of all when you're going to make a film you must Decide if its a thriller or a drama! As a dram

# Experimential setup

In [8]:
def experimental_setup(df):
    # Split into X and y
    X = df['review']
    y = df['sentiment']
    
    # Split data into train and test set
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train on full data
    X_train = X_train
    y_train = y_train

    # Transform text reviews to Bag of Words representation
    X_train = vectorizer.fit_transform(X_train)
    X_test = vectorizer.transform(X_test)
    
    # Print the shapes of the sets
    print(f"Training Set: X_train shape = {X_train.shape}, y_train shape = {y_train.shape}")
    print(f"Test Set: X_test shape = {X_test.shape}, y_test shape = {y_test.shape}")
    print()

    return X_train, y_train, X_test, y_test

# Modeling and Performance metrics

In [9]:
# Function to train and evaluate models with multiple metrics
def evaluate_models_with_metrics(models, X_train, y_train, X_test, y_test):
    results = []

    for name, model in models:      
        # Train the model
        model.fit(X_train, y_train)
        
        # Predict on test set
        y_test_pred = model.predict(X_test)

        # Store results for the model
        model_results = {
            'Model': name,
            'Test Accuracy': accuracy_score(y_test, y_test_pred),
            'Test Precision': precision_score(y_test, y_test_pred),
            'Test Recall': recall_score(y_test, y_test_pred),
            'Test F1-Score': f1_score(y_test, y_test_pred)
        }
        
        results.append(model_results)

    # Convert results to a pandas DataFrame
    results_df = pd.DataFrame(results)
    
    return results_df

In [10]:
# Create the split
X_train_base, y_train_base, X_test_base, y_test_base = experimental_setup(base_df)

# We make the test set as big as the number of classifcations done by the LLM
X_test_base = X_test_base[:n_rows]
y_test_base = y_test_base[:n_rows]

# Evaluate models with multiple metrics and print results
base_results = evaluate_models_with_metrics(ml_models, X_train_base, y_train_base, X_test_base, y_test_base)
display(base_results)

Training Set: X_train shape = (40000, 10000), y_train shape = (40000,)
Test Set: X_test shape = (10000, 10000), y_test shape = (10000,)



,Model,Test Accuracy,Test Precision,Test Recall,Test F1-Score
0,Logistic Regression,0.8700,0.869514,0.872965,0.871236
1,Random Forest,0.8588,0.866559,0.850734,0.858574
2,SVM,0.8798,0.858103,0.912267,0.884356
3,KNN,0.6062,0.615449,0.581977,0.598245
4,Gradient Boosting,0.8140,0.784869,0.868996,0.824793


# LLM Classification

### Qualitative loop

In [11]:
prompt = '''Classify this review as positive (1) or negative (0). Return ONLY "1" or "0", without any additional text.

Review:
{review}'''

In [12]:
# Initialize a dictionary to store results per model
model_list = {}

# Classify on the same test set
_, X_test, _, y_test = train_test_split(base_df['review'], base_df['sentiment'], test_size=0.2, random_state=42)
llm_test_df = pd.DataFrame({'review': X_test, 'sentiment': y_test})

# Iterate through models
for index, model in enumerate(models):
    print("Processing Model: " + model + " (Model " + str(index + 1) + "/" + str(len(models)) + ")")
    model_list[model] = []

    # Classify n_rows
    for index, row in tqdm(llm_test_df[:n_rows].iterrows(), total=n_rows, desc="Classifying rows"):
        # Format reponse based on row
        formatted_prompt = prompt.format(review=row['review'])

        attempts = 0
        raw_response = None

        # Try maximum of 10 times
        while attempts < 10:
            try:
                raw_response = ollama.generate(model=model, prompt=formatted_prompt)['response']
                raw_response = raw_response 

                response = int(raw_response)
                assert response in [0, 1]

                model_list[model].append([response])
                break # Successful classification, exit the loop

            except Exception as e:
                # Correct data
                lower_raw_response = raw_response.lower()
                
                # Switch
                if 'positive' in lower_raw_response and 'negative' not in lower_raw_response:
                    model_list[model].append([1])
                    break # Successful classification, exit the loop
                elif 'positive' not in lower_raw_response and 'negative' in lower_raw_response:
                    model_list[model].append([0])
                    break # Successful classification, exit the loop
                elif '1' in lower_raw_response and '0' not in lower_raw_response:
                    model_list[model].append([1])
                    break # Successful classification, exit the loop
                elif '1' not in lower_raw_response and '0' in lower_raw_response:
                    model_list[model].append([0])
                    break # Successful classification, exit the loop

                # Not succesful try again
                attempts += 1

        if attempts == 10:
            print("Sample unknown")
            model_list[model].append([random.randint(0, 1)])

Processing Model: llama3.2:1b (Model 1/8)


Classifying rows: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5000/5000 [15:39<00:00,  5.32it/s]


Processing Model: llama3.2:3b (Model 2/8)


Classifying rows: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5000/5000 [35:49<00:00,  2.33it/s]


Processing Model: gemma3:1b (Model 3/8)


...

Classifying rows: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5000/5000 [1:27:26<00:00,  1.05s/it]


Processing Model: mistral (Model 7/8)


Classifying rows: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5000/5000 [1:12:11<00:00,  1.15it/s]


Processing Model: deepseek-llm (Model 8/8)


Classifying rows: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5000/5000 [2:16:33<00:00,  1.64s/it]


# Comparison

In [13]:
def compare(base_df, llm_df, model_name):
    # Select relevant evaluation metrics
    # Set 'Model' as the index in DataFrames
    base_df_results = base_df.set_index('Model')
    llm_df_results = llm_df.set_index('Model')
    
    # Rename evaluation metrics columns
    base_df_results = base_df_results.rename(columns={
        'Test Accuracy': 'Accuracy', 'Test Precision': 'Precision',
        'Test Recall': 'Recall', 'Test F1-Score': 'F1-Score'
    })
    
    llm_df_results = llm_df_results.rename(columns={
        'Test Accuracy': 'Accuracy', 'Test Precision': 'Precision',
        'Test Recall': 'Recall', 'Test F1-Score': 'F1-Score'
    })

    # Merge dfs
    raw_final_results = pd.concat([base_df_results, llm_df_results])

    # Show raw data results
    print("Raw performance metrics:")
    display(raw_final_results)
    
    # Calculate and display the difference matrix (% difference)
    df_diff = round((raw_final_results.loc[model_name] - raw_final_results) / raw_final_results.loc[model_name] * 100, 1)
    
    # Drop the reference model row (since its diff would be 0)
    df_diff = df_diff.drop(index=model_name)
    
    # Display the result
    print("% Difference between ML models and LLM classifcations:")
    display(df_diff)
    print()

In [14]:
# Iterate through models
for index, model in enumerate(models):
    print("Processing Model: " + model + " (Model " + str(index + 1) + "/" + str(len(models)) + ")")

    # Get data and make a DataFrame
    llm_classification_df = pd.DataFrame(model_list[model], columns=['sentiment'])
    
    # y's
    # Remember the true y is the same y from the test set that we classified earlier
    y_true = llm_test_df['sentiment'][:n_rows]
    llm_y_pred = llm_classification_df['sentiment']

    # Store results for the classifications as df
    llm_performance_metrics = pd.DataFrame([{
            'Model': model,
            'Test Accuracy': accuracy_score(y_true, llm_y_pred),
            'Test Precision': precision_score(y_true, llm_y_pred),
            'Test Recall': recall_score(y_true, llm_y_pred),
            'Test F1-Score': f1_score(y_true, llm_y_pred)
    }])

    # Compare the results between base and LLM altered df
    compare(base_results, llm_performance_metrics, model)

Processing Model: llama3.2:1b (Model 1/8)
Raw performance metrics:


,Accuracy,Precision,Recall,F1-Score
Model,,,,
Logistic Regression,0.8700,0.869514,0.872965,0.871236
Random Forest,0.8588,0.866559,0.850734,0.858574
SVM,0.8798,0.858103,0.912267,0.884356
KNN,0.6062,0.615449,0.581977,0.598245
Gradient Boosting,0.8140,0.784869,0.868996,0.824793
llama3.2:1b,0.6056,0.662895,0.441842,0.530253


% Difference between ML models and LLM classifcations:


,Accuracy,Precision,Recall,F1-Score
Model,,,,
Logistic Regression,-43.7,-31.2,-97.6,-64.3
Random Forest,-41.8,-30.7,-92.5,-61.9
SVM,-45.3,-29.4,-106.5,-66.8
KNN,-0.1,7.2,-31.7,-12.8
Gradient Boosting,-34.4,-18.4,-96.7,-55.5



Processing Model: llama3.2:3b (Model 2/8)
Raw performance metrics:


...

,Accuracy,Precision,Recall,F1-Score
Model,,,,
Logistic Regression,7.6,6.9,8.3,7.6
Random Forest,8.8,7.2,10.6,9.0
SVM,6.6,8.1,4.2,6.2
KNN,35.6,34.1,38.9,36.6
Gradient Boosting,13.6,16.0,8.7,12.5



Processing Model: deepseek-llm (Model 8/8)
Raw performance metrics:


,Accuracy,Precision,Recall,F1-Score
Model,,,,
Logistic Regression,0.8700,0.869514,0.872965,0.871236
Random Forest,0.8588,0.866559,0.850734,0.858574
SVM,0.8798,0.858103,0.912267,0.884356
KNN,0.6062,0.615449,0.581977,0.598245
Gradient Boosting,0.8140,0.784869,0.868996,0.824793
deepseek-llm,0.6296,0.576543,0.997221,0.730657


% Difference between ML models and LLM classifcations:


,Accuracy,Precision,Recall,F1-Score
Model,,,,
Logistic Regression,-38.2,-50.8,12.5,-19.2
Random Forest,-36.4,-50.3,14.7,-17.5
SVM,-39.7,-48.8,8.5,-21.0
KNN,3.7,-6.7,41.6,18.1
Gradient Boosting,-29.3,-36.1,12.9,-12.9
